In [1]:
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Creating the Connections

In [2]:
# create the connection to MongoDB
# create an instance of the database and collections

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.australia_fire_db
bushfiresbyState = db.bushfiresbyState
historicalFires = db.historicalFires

In [15]:
# urls to scrape

byStateurl = 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Australian_bushfire_season'
historicalurl = 'https://en.wikipedia.org/wiki/List_of_major_bushfires_in_Australia'

In [16]:
# request the html using beautiful soup

historical_response = requests.get(historicalurl)
bystate_response = requests.get(byStateurl)

In [17]:
# parse the html text

h_soup = BeautifulSoup(historical_response.text, 'html.parser')
bs_soup = BeautifulSoup(bystate_response.text, 'html.parser')

# Historical Bushfire Scrapping

In [ ]:
# the table headers are within the table body in the first two rows

h_table = h_soup.find('table', class_="wikitable")
h_table_headers = h_soup.find_all("tr")[0:2]

In [ ]:
# scrape the table headers and append them to the headers list

h_headers = []
for table_header in h_table_headers:
    try:
        value = table_header.text
#         value = value.split("\n")
        h_headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(h_headers)
print(len(h_headers))

In [ ]:
# manually create the headers list, because the scraped table headers was too complicated
# if there is time, create logic to put the two header rows into one similar to the list below

h_headers = ['Date', 'Name', 'State(s)/territories', 'AreaBurned(ha)', 'AreaBurned(acres)', 'Fatalities', 'PropertiesDamaged(HomesDestroyed)']

In [ ]:
# find objects to scrape
# the table data is after the headers which is in the first two rows

h_table = h_soup.find('table', class_="wikitable")
h_table_body = h_table.find("tbody")
h_table_row = h_table_body.find_all('tr')[2:]

In [ ]:
# scrape data and create a list of lists for each row of data

h_data = []

for row in h_table_row:
    
    datarow = []
    
    table_data = row.find_all('td')
    
    for tdata in table_data:
        try:
            value = tdata.text
    #             value.split("\n\n")
            value = value.replace("\n", "")
            datarow.append(value)

        except AttributeError as e:
            print(e)
            
    h_data.append(datarow)
        
# print(h_data)

In [ ]:
# convert scrapped data into a dataframe

h_df = pd.DataFrame(h_data)
h_df

In [ ]:
# keep desired rows

h_drop_rows_df = h_df[[0,1,2,3,4,5,6]]
h_drop_rows_df

In [ ]:
# Add column headers

h_drop_rows_df.columns = h_headers
h_drop_rows_df

In [ ]:
# replace Nil with 0 values

h_nil_df = h_drop_rows_df.replace('Nil', '0')
h_nil_df.head(10)

In [ ]:
# remove commas from numbers
# replace unwanted values as NaNs
# replace range data into an average value
# remove citations found within []

h_tonumeric_df = h_nil_df.copy()
h_tonumeric_df['AreaBurned(ha)'] = h_nil_df['AreaBurned(ha)'].str.replace(',','')
h_tonumeric_df['AreaBurned(acres)'] = h_nil_df['AreaBurned(acres)'].str.replace(',','')
h_tonumeric_df['PropertiesDamaged(HomesDestroyed)'] = h_nil_df['PropertiesDamaged(HomesDestroyed)'].str.replace(',','')

columnstoedit = ['AreaBurned(ha)','AreaBurned(acres)','Fatalities','PropertiesDamaged(HomesDestroyed)']

for i in range(len(h_tonumeric_df['Date'])):
    for column in columnstoedit:
        
        if ((h_tonumeric_df[column][i] == '') and (h_tonumeric_df[column][i] != '0')) or (h_tonumeric_df[column][i] == 'unknown') :
            h_tonumeric_df[column][i] = 'NaN'

        if 'approx. ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].replace('approx. ', '')
            
        if 'than ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].split(' ')[-1]

        if len(str(h_tonumeric_df[column][i]).split('–')) == 2:
            splitvalues = str(h_tonumeric_df[column][i]).split('–')
            h_tonumeric_df[column][i] = np.mean([int(splitvalues[0]), int(splitvalues[1])])

        if len(str(h_tonumeric_df[column][i]).split('[')) >= 2:
            h_tonumeric_df[column][i] = str(h_tonumeric_df[column][i]).split('[')[0]
        

h_tonumeric_df

In [ ]:
# pull out the year the fire took place

h_year_df = h_tonumeric_df.copy()
h_year_df['Year'] = ''
for i in range(len(h_year_df['Date'])):

    if len(str(h_year_df['Date'][i]).split(' – ')) >= 2:
        results = str(h_year_df['Date'][i]).split(' – ')
        for result in results:
            if len(str(result).split()) == 4:
                year = result
    
    elif len(str(h_year_df['Date'][i]).split('[')) >= 2:
        year = str(h_year_df['Date'][i]).split('[')[0]
        
#     elif len(str(h_year_df['Date'][i]).split(' – ')) >= 2:
#         results = str(h_year_df['Date'][i]).split(' – ')
#         for result in results:
#             if len(str(result).split()) == 4:
#                 year = result
    
    else:
        year = str(h_year_df['Date'][i]).split(' ')[-1]
        
    h_year_df['Year'][i] = year

    
h_year_df

In [ ]:
# remove nan values and empty values and convert them to integers

h_casting_df = h_year_df.copy()

h_casting_df = h_casting_df[h_casting_df['AreaBurned(ha)'] != 'NaN']
h_casting_df = h_casting_df[h_casting_df['PropertiesDamaged(HomesDestroyed)'] != 'NaN']
h_casting_df = h_casting_df[h_casting_df['PropertiesDamaged(HomesDestroyed)'] != '']
h_casting_df = h_casting_df.astype(
    {
        'AreaBurned(ha)':'int',
        'AreaBurned(acres)': 'int',
        'Fatalities':'int',
        'PropertiesDamaged(HomesDestroyed)':'int',
        'Year': 'int'
    })
# h_casting_df['Year'] = pd.to_datetime(h_casting_df['Year'], format = '%Y')
h_casting_df.dtypes
h_casting_df

In [ ]:
# transform the dataframe into a dictionary so that we can feed it to MongoDB
h_dict = h_casting_df.to_dict('records')
h_dict

In [ ]:
# insert records into the MongoDB collection histroicalFires

if (historicalFires.count() == 0):
    historicalFires.insert(h_dict)
    
else:
    print("Data already exists")

# Scrapping Data from 2019-2020 by State

In [22]:
bs_table = bs_soup.find('table', class_='sortable')
print(bs_table.prettify())

<table class="wikitable sortable plainrowheaders">
 <tbody>
  <tr>
   <th rowspan="2" scope="col" style="font-weight:bold;">
    State / territory
   </th>
   <th rowspan="2" scope="col" style="font-weight:bold;">
    Fatalities
   </th>
   <th rowspan="2" scope="col" style="font-weight:bold;">
    Homes
    <br/>
    lost
   </th>
   <th colspan="2" scope="col" style="font-weight:bold;">
    Area
    <br/>
    <span style="font-size:85%;">
     (estimated)
    </span>
   </th>
   <th rowspan="2" scope="col" style="font-weight:bold;">
    Notes
   </th>
  </tr>
  <tr>
   <th>
    <a href="/wiki/Hectare" title="Hectare">
     ha
    </a>
   </th>
   <th>
    <a href="/wiki/Acre" title="Acre">
     acres
    </a>
   </th>
  </tr>
  <tr>
   <th scope="row">
    Australian Capital Territory
   </th>
   <td align="right">
    1
   </td>
   <td align="right">
    0
   </td>
   <td style="text-align:right;">
    56,688
   </td>
   <td style="text-align:right;">
    140,080
   </td>
   <td>
  

In [56]:
bs_theaders = bs_soup.find('table', class_='sortable').find_all('tr')[0:2]


bs_headers = []
for table_header in bs_theaders:
    try:
        value = table_header.text
        value = value.split("\n\n")
        bs_headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(bs_headers)
print(len(bs_headers))

[['\nState / territory', 'Fatalities', 'Homeslost', 'Area(estimated)', 'Notes\n'], ['\nha\nacres\n']]
2


In [64]:
bs_theaders = ['State/Territory', 'Fatalities', 'Homeslost', 'Area(estimated)(ha)', 'Area(estimated)(acres)', 'Notes']

In [70]:
# scrape data and create a list of lists for each row of data

bs_table_row = bs_soup.find('table', class_='sortable').find_all('tr')[2:]

bs_data = []

for row in bs_table_row:
    
    datarow = []
    
    table_header = row.find('th').text
    table_header = table_header.replace('\n','')
    datarow.append(table_header)
    table_data = row.find_all('td')
    
    for tdata in table_data:
        try:
            value = tdata.text
    #             value.split("\n\n")
            value = value.replace("\n", "")
            datarow.append(value)

        except AttributeError as e:
            print(e)
            
    bs_data.append(datarow)
        
print(bs_data)

[['Australian Capital Territory', '1', '0', '56,688', '140,080', 'Area;[91][92][93] fatality[c][95]'], ['New South Wales', '25', '2,439', '5,400,000', '13,300,000', 'Area;[96] fatalities;[18][37] homes[96]'], ['Northern Territory', '0', '5', '6,800,000', '16,800,000', 'Area, includes mainly scrub fires, which are within the normal range of area burnt by bushfires each year;[49] homes[97]'], ['Queensland', '0', '48', '2,500,000', '6,180,000', 'Area, includes scrub fires;[49] homes[97][d]'], ['South Australia', '3', '151', '490,000', '1,210,000', 'Area;[49] fatalities;[102] homes (KI:65)[103] (AH:86)[104]'], ['Tasmania', '0', '2', '36,000', '89,000', 'Area;[49] homes[97]'], ['Victoria', '5', '396', '1,500,000', '3,710,000', 'Area;[49] fatalities;[15] homes[105]'], ['Western Australia', '0', '1', '2,200,000', '5,440,000', 'Area, includes scrub fires;[49] homes[97]'], ['Total', '34', '3,500+', '18,736,070', '46,300,000', '[e][b][109][110] Total area estimate as of 13 February 2020; current

In [71]:
# convert scrapped data into a dataframe

bs_df = pd.DataFrame(bs_data)
bs_df

,0,1,2,3,4,5
0,Australian Capital Territory,1,0,"56,688","140,080",Area;[91][92][93] fatality[c][95]
1,New South Wales,25,"2,439","5,400,000","13,300,000",Area;[96] fatalities;[18][37] homes[96]
2,Northern Territory,0,5,"6,800,000","16,800,000","Area, includes mainly scrub fires, which are w..."
3,Queensland,0,48,"2,500,000","6,180,000","Area, includes scrub fires;[49] homes[97][d]"
4,South Australia,3,151,"490,000","1,210,000",Area;[49] fatalities;[102] homes (KI:65)[103] ...
5,Tasmania,0,2,"36,000","89,000",Area;[49] homes[97]
6,Victoria,5,396,"1,500,000","3,710,000",Area;[49] fatalities;[15] homes[105]
7,Western Australia,0,1,"2,200,000","5,440,000","Area, includes scrub fires;[49] homes[97]"
8,Total,34,"3,500+","18,736,070","46,300,000",[e][b][109][110] Total area estimate as of 13 ...


In [72]:
# Add column headers

bs_df.columns = bs_theaders
bs_df

,State/Territory,Fatalities,Homeslost,Area(estimated)(ha),Area(estimated)(acres),Notes
0,Australian Capital Territory,1,0,"56,688","140,080",Area;[91][92][93] fatality[c][95]
1,New South Wales,25,"2,439","5,400,000","13,300,000",Area;[96] fatalities;[18][37] homes[96]
2,Northern Territory,0,5,"6,800,000","16,800,000","Area, includes mainly scrub fires, which are w..."
3,Queensland,0,48,"2,500,000","6,180,000","Area, includes scrub fires;[49] homes[97][d]"
4,South Australia,3,151,"490,000","1,210,000",Area;[49] fatalities;[102] homes (KI:65)[103] ...
5,Tasmania,0,2,"36,000","89,000",Area;[49] homes[97]
6,Victoria,5,396,"1,500,000","3,710,000",Area;[49] fatalities;[15] homes[105]
7,Western Australia,0,1,"2,200,000","5,440,000","Area, includes scrub fires;[49] homes[97]"
8,Total,34,"3,500+","18,736,070","46,300,000",[e][b][109][110] Total area estimate as of 13 ...


In [87]:
bs_dropchar_df = bs_df.drop('Notes', axis=1)

columnstoedit = ['Homeslost', 'Area(estimated)(ha)', 'Area(estimated)(acres)']

for column in columnstoedit:
    for i in range(len(bs_dropchar_df[column])):
        try:
            bs_dropchar_df[column][i] = str(bs_dropchar_df[column][i]).replace(',','')
        except:
            pass

bs_dropchar_df

,State/Territory,Fatalities,Homeslost,Area(estimated)(ha),Area(estimated)(acres)
0,Australian Capital Territory,1,0,56688,140080
1,New South Wales,25,2439,5400000,13300000
2,Northern Territory,0,5,6800000,16800000
3,Queensland,0,48,2500000,6180000
4,South Australia,3,151,490000,1210000
5,Tasmania,0,2,36000,89000
6,Victoria,5,396,1500000,3710000
7,Western Australia,0,1,2200000,5440000
8,Total,34,3500+,18736070,46300000
